# **EA2: Practico**

## **Problema 1**

Pokemon ha sido un juego de video y cartas que ha estado en el radar de muchos jóvenes y adultos, al grado que actualmente se ha visto un auge en la versión competitiva en los juegos de video. Un pokemon cuenta con diferentes características que lo hacen único, entre ellas se encuentran el tipo de pokemon, el ataque, la defensa, la velocidad, la vida, entre otros. En la versión competitiva se busca tener un equipo balanceado que pueda enfrentar a cualquier tipo de pokemon, por lo que se busca tener un equipo que tenga un balance entre ataque, defensa, velocidad y vida. Otros elementos que se buscan en un equipo son el tipo de pokemon, ya que algunos tipos son más fuertes que otros, por lo que se busca tener un equipo que pueda enfrentar a cualquier tipo de pokemon, el cual tiene tipos principales y secundarios

### **Analisis del Problema**

Resumiendo el problema que nos es presentado, tenemos que encontrar un equipo de pokemon optimo el cual tenga un balance en sus estadisticas de ataque, defensa, velocidad y vida. Como no es mencionado en el problema, asumiremos que el ataque y la defensa es el promedio de las estadisticas de ataque fisico  y especial, al igual que defensa fisica y especial para poder usar la mayor cantidad de datos posibles de los pokemones, esto se convertira de vuelta a int ya que no hay estadisticas floats.
Con esto, tambien tendremos que formar un equipo balanceado en cuanto a tipos, esto para que cubra una mayor area de ataque y defensa a otros equipos.

### **Preguntas**

Las preguntas que se nos pueden plantear en la resolucion de este problema son, 

¿El equipo debe tener solo pokemones con altas estadisticas totales, o especificas?

¿Que equipo cubre la mayor cantidad de tipos contra los que es efectivo?

¿Cuantos de los pokemones seran legendarios, o importara en el momento de agrupar el equipo?

¿Sera mas importante tener pokemones de 1 solo tipo o de 2, o dependera de estadisticas?

¿Que generacion de pokemones tiene los mas fuertes?

**Cargar librerias**

In [917]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import kmapper as km
import math
import warnings
import sklearn
from sklearn.decomposition import PCA

np.random.seed(87)
warnings.simplefilter("ignore")

In [918]:
df = pd.read_csv("../Datos/Pokemon.csv")

In [919]:
# Agregar promedio de attack y special attack, al igual don defense
df["avg_attack"] = (df["Attack"] + df["Sp. Atk"]) / 2
df["avg_defense"] = (df["Defense"] + df["Sp. Def"]) / 2
# Redondear los valores
df["avg_attack"] = df["avg_attack"].apply(lambda x: math.ceil(x))
df["avg_defense"] = df["avg_defense"].apply(lambda x: math.ceil(x))

### **Analisis Exploratorio**

In [920]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   #            800 non-null    int64 
 1   Name         800 non-null    object
 2   Type 1       800 non-null    object
 3   Type 2       414 non-null    object
 4   Total        800 non-null    int64 
 5   HP           800 non-null    int64 
 6   Attack       800 non-null    int64 
 7   Defense      800 non-null    int64 
 8   Sp. Atk      800 non-null    int64 
 9   Sp. Def      800 non-null    int64 
 10  Speed        800 non-null    int64 
 11  Generation   800 non-null    int64 
 12  Legendary    800 non-null    bool  
 13  avg_attack   800 non-null    int64 
 14  avg_defense  800 non-null    int64 
dtypes: bool(1), int64(11), object(3)
memory usage: 88.4+ KB


In [921]:
df.describe()

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,avg_attack,avg_defense
count,800.000000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000,800.000000,800.000000
mean,362.813750,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375,76.100000,73.013750
std,208.343798,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129,27.220786,25.661044
min,1.000000,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000,10.000000,18.000000
25%,184.750000,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000,56.000000,55.000000
50%,364.500000,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000,73.500000,70.000000
75%,539.250000,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000,91.000000,90.000000
max,721.000000,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000,180.000000,230.000000


In [922]:
# Arreglamos los valores nulos del dataframe
df["Type 2"] = df["Type 2"].fillna("None")

In [923]:
# Creamos un dataframe numerico
df_num = df.select_dtypes(include=[np.number])
df_num.drop(
    ["Attack", "Generation", "Defense", "Sp. Atk", "Sp. Def"],
    axis=1,
    inplace=True,
)

In [924]:
# Creamos una grafica con el avg attack, avg defense, vida y velocidad a traves de los ids
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=df["#"], y=df["avg_attack"], mode="markers", name="avg_attack")
)
fig.add_trace(
    go.Scatter(x=df["#"], y=df["avg_defense"], mode="markers", name="avg_defense")
)
fig.add_trace(go.Scatter(x=df["#"], y=df["HP"], mode="markers", name="HP"))
fig.add_trace(go.Scatter(x=df["#"], y=df["Speed"], mode="markers", name="Speed"))
fig.update_layout(
    title="Pokemon Stats", xaxis_title="Pokemon ID", yaxis_title="Stat Value"
)
fig.show()

In [925]:
df_num.corr()

,#,Total,HP,Speed,avg_attack,avg_defense
#,1.000000,0.119813,0.097614,0.010733,0.113933,0.102420
Total,0.119813,1.000000,0.618748,0.575943,0.887500,0.760963
HP,0.097614,0.618748,1.000000,0.175952,0.469565,0.351034
Speed,0.010733,0.575943,0.175952,1.000000,0.511516,0.149124
avg_attack,0.113933,0.887500,0.469565,0.511516,1.000000,0.489247
avg_defense,0.102420,0.760963,0.351034,0.149124,0.489247,1.000000


In [926]:
# Sacamos los maximos de las estadisticas que se piden y las graficamos
hp = df_num["HP"].idxmax()
atk = df_num["avg_attack"].idxmax()
dfnse = df_num["avg_defense"].idxmax()
speed = df_num["Speed"].idxmax()
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["#"], y=df["HP"], mode="markers", name="HP"))
fig.add_trace(
    go.Scatter(x=df.index, y=df["avg_attack"], mode="markers", name="avg_attack")
)
fig.add_trace(
    go.Scatter(x=df.index, y=df["avg_defense"], mode="markers", name="avg_defense")
)
fig.add_trace(go.Scatter(x=df.index, y=df["Speed"], mode="markers", name="Speed"))
fig.update_layout(
    title="Pokemon Stats", xaxis_title="Pokemon ID", yaxis_title="Stat Value"
)
fig.add_trace(
    go.Scatter(
        x=[hp],
        y=[df["HP"][hp]],
        mode="markers",
        name="Max HP",
        marker=dict(color="red", size=10),
    )
)
fig.add_trace(
    go.Scatter(
        x=[atk],
        y=[df["avg_attack"][atk]],
        mode="markers",
        name="Max Attack",
        marker=dict(color="red", size=10),
    )
)
fig.add_trace(
    go.Scatter(
        x=[dfnse],
        y=[df["avg_defense"][dfnse]],
        mode="markers",
        name="Max Defense",
        marker=dict(color="red", size=10),
    )
)
fig.add_trace(
    go.Scatter(
        x=[speed],
        y=[df["Speed"][speed]],
        mode="markers",
        name="Max Speed",
        marker=dict(color="red", size=10),
    )
)
fig.show()

In [927]:
# Sacamos el nombre de los pokemones de los maximos desde un df nuevo con las columnas que no tienen el df numerico
df_car = df[["#", "Name", "Type 1", "Type 2", "Legendary", "Generation"]]
df_car.loc[[hp, atk, dfnse, speed]]

,#,Name,Type 1,Type 2,Legendary,Generation
261,242,Blissey,Normal,None,False,2
426,384,RayquazaMega Rayquaza,Dragon,Flying,True,3
230,213,Shuckle,Bug,Rock,False,2
431,386,DeoxysSpeed Forme,Psychic,None,True,3


Con este primer analisis podemos ver que solo 2 de los pokemones con estadisticas mas altas son legendarios, y tambien solo tienen 1 tipo 2 de estos. Dividiendolo en mitad y mitad. Esto nos da un espectro para contestar las ultimas 2 preguntas planteadas, ya que vemos que un equipo no necesita pokemones legendarios necesariamente para que sea bueno. Tambien nos damos cuenta que muchos pokemones buenos van a tener solo un tipo, por lo que se necesitara hacer un analisis, o por preferencia de si mantener un pokemon con 2 tipos o con mayores estadisticas.

Tambien podemos contestarnos de una manera no tan informada la 3er pregunta, ya que en estos pokemones vemos que los de mayor estadisticas son de la generacion 2 y 3.

In [928]:
# Hacer un diccionario de los tipos de pokemones y poner los numeros en un dataframe nuevo con solo las columnas de interes
df2 = df[
    [
        "#",
        "Type 1",
        "Type 2",
        "Total",
        "HP",
        "avg_attack",
        "avg_defense",
        "Speed",
    ]
]
type_dict = {}
for i in df2["Type 1"].unique():
    type_dict[i] = len(type_dict)
for i in df2["Type 2"].unique():
    if i not in type_dict:
        type_dict[i] = len(type_dict)
for key in type_dict:
    type_dict[key] += 1
type_dict["None"] = 0
df2["Type 1"] = df2["Type 1"].apply(lambda x: type_dict[x])
df2["Type 2"] = df2["Type 2"].apply(lambda x: type_dict[x])
df2.head()

,#,Type 1,Type 2,Total,HP,avg_attack,avg_defense,Speed
0,1,1,6,318,45,57,57,45
1,2,1,6,405,60,71,72,60
2,3,1,6,525,80,91,92,80
3,3,1,6,625,80,111,122,80
4,4,2,0,309,39,56,47,65


### **Diccionario de tipos**

{'Grass': 1,
 'Fire': 2,
 'Water': 3,
 'Bug': 4,
 'Normal': 5,
 'Poison': 6,
 'Electric': 7,
 'Ground': 8,
 'Fairy': 9,
 'Fighting': 10,
 'Psychic': 11,
 'Rock': 12,
 'Ghost': 13,
 'Ice': 14,
 'Dragon': 15,
 'Dark': 16,
 'Steel': 17,
 'Flying': 18,
 'None': 0}

Para poder incluir la fuerza y debilidad de los tipos de los pokemones, agregaremos un diccionario con una proporcion de efectividad en ataque, dividiendo esto entre la efectividad maxima la cual seria sun tipo super efectivo contra todo, esto se hara tambien con la defensa.

Esto se hara con el proposito de sacar valores mas reales de ataque y defensa con respecto a su tipo, incluyendo ambos tipos, ya que las proporciones de ambos tipos se multiplicaran.

**Ataque**

In [929]:
# En ataque si el tipo es muy efectivo se suma 2, si es poco efectivo se suma 0.5, y si no afecta se suma 0, y si afecta normal se suma 1
attack_sum = {
    "Grass": (7 * 0.5 + 3 * 2 + 8) / 36,
    "Fire": (4 * 0.5 + 4 * 2 + 10) / 36,
    "Water": (3 * 0.5 + 3 * 2 + 12) / 36,
    "Bug": (7 * 0.5 + 3 * 2 + 8) / 36,
    "Normal": (2 * 0.5 + 0 * 2 + 16) / 36,
    "Poison": (7 * 0.5 + 3 * 2 + 7) / 36,
    "Electric": (3 * 0.5 + 2 * 2 + 12) / 36,
    "Ground": (2 * 0.5 + 5 * 2 + 10) / 36,
    "Fairy": (3 * 0.5 + 3 * 2 + 12) / 36,
    "Fighting": (2 * 0.5 + 4 * 2 + 12) / 36,
    "Psychic": (2 * 0.5 + 2 * 2 + 13) / 36,
    "Rock": (3 * 0.5 + 4 * 2 + 11) / 36,
    "Ghost": (1 * 0.5 + 2 * 2 + 14) / 36,
    "Ice": (4 * 0.5 + 4 * 2 + 10) / 36,
    "Dragon": (1 * 0.5 + 1 * 2 + 15) / 36,
    "Dark": (3 * 0.5 + 2 * 2 + 13) / 36,
    "Steel": (4 * 0.5 + 3 * 2 + 11) / 36,
    "Flying": (3 * 0.5 + 3 * 2 + 12) / 36,
    "None": 0,
}

**Defensa**

In [930]:
# En defensa si resiste a un tipo se suma 2, si es debil se suma 0.5, si es inmune se suma 2 y si es normal se suma 1
defense_sum = {
    "Grass": (5 * 0.5 + 4 * 2 + 9) / 36,
    "Fire": (3 * 0.5 + 5 * 2 + 13) / 36,
    "Water": (2 * 0.5 + 4 * 2 + 12) / 36,
    "Bug": (3 * 0.5 + 3 * 2 + 12) / 36,
    "Normal": (1 * 0.5 + 1 * 2 + 16 + 1) / 36,
    "Poison": (2 * 0.5 + 5 * 2 + 11) / 36,
    "Electric": (1 * 0.5 + 3 * 2 + 14) / 36,
    "Ground": (3 * 0.5 + 2 * 2 + 13 + 1) / 36,
    "Fairy": (2 * 0.5 + 3 * 2 + 13 + 1) / 36,
    "Fighting": (3 * 0.5 + 3 * 2 + 12) / 36,
    "Psychic": (3 * 0.5 + 2 * 2 + 13) / 36,
    "Rock": (5 * 0.5 + 4 * 2 + 9) / 36,
    "Ghost": (2 * 0.5 + 2 * 2 + 14 + 2) / 36,
    "Ice": (4 * 0.5 + 1 * 2 + 13) / 36,
    "Dragon": (3 * 0.5 + 4 * 2 + 11) / 36,
    "Dark": (2 * 0.5 + 3 * 2 + 13 + 1) / 36,
    "Steel": (3 * 0.5 + 10 * 2 + 5 + 1) / 36,
    "Flying": (3 * 0.5 + 3 * 2 + 12 + 1) / 36,
    "None": 0,
}

In [931]:
# El coeficiente de ataque y defensa se aplica dependiendo al tipo
df2["Attack Type 1"] = df["Type 1"].apply(lambda x: attack_sum[x])


df2["Attack Type 2"] = df["Type 2"].apply(lambda x: attack_sum[x])


df2["Defense Type 1"] = df["Type 1"].apply(lambda x: defense_sum[x])


df2["Defense Type 2"] = df["Type 2"].apply(lambda x: defense_sum[x])


df2.head(10)

,#,Type 1,Type 2,Total,HP,avg_attack,avg_defense,Speed,Attack Type 1,Attack Type 2,Defense Type 1,Defense Type 2
0,1,1,6,318,45,57,57,45,0.486111,0.458333,0.541667,0.611111
1,2,1,6,405,60,71,72,60,0.486111,0.458333,0.541667,0.611111
2,3,1,6,525,80,91,92,80,0.486111,0.458333,0.541667,0.611111
3,3,1,6,625,80,111,122,80,0.486111,0.458333,0.541667,0.611111
4,4,2,0,309,39,56,47,65,0.555556,0.000000,0.680556,0.000000
5,5,2,0,405,58,72,62,80,0.555556,0.000000,0.680556,0.000000
6,6,2,18,534,78,97,82,100,0.555556,0.541667,0.680556,0.569444
7,6,2,15,634,78,130,98,100,0.555556,0.486111,0.680556,0.569444
8,6,2,18,634,78,132,97,100,0.555556,0.541667,0.680556,0.569444
9,7,3,0,314,44,49,65,43,0.541667,0.000000,0.583333,0.000000


Sumamos los 2 tipos que existen del pokemon, favoreciendo los que tienen 2 tipos ya que cubren mas efectividades. 

In [932]:
df2["Attack Type"] = df2["Attack Type 1"] + df2["Attack Type 2"]
df2["Defense Type"] = df2["Defense Type 1"] + df2["Defense Type 2"]
df2.drop(
    ["Attack Type 1", "Attack Type 2", "Defense Type 1", "Defense Type 2"],
    axis=1,
    inplace=True,
)
df2.head(5)

,#,Type 1,Type 2,Total,HP,avg_attack,avg_defense,Speed,Attack Type,Defense Type
0,1,1,6,318,45,57,57,45,0.944444,1.152778
1,2,1,6,405,60,71,72,60,0.944444,1.152778
2,3,1,6,525,80,91,92,80,0.944444,1.152778
3,3,1,6,625,80,111,122,80,0.944444,1.152778
4,4,2,0,309,39,56,47,65,0.555556,0.680556


Ajustamos el ataque y defensa con su tipo, esto hace que su tipo este afectando sus estadisticas generales, especificamente ataque y defensa, y total.

In [933]:
df2["adj_attack"] = df2["avg_attack"] * df2["Attack Type"]
df2["adj_defense"] = df2["avg_defense"] * df2["Defense Type"]
df2["adj_attack"] = df2["adj_attack"].apply(lambda x: math.ceil(x))
df2["adj_defense"] = df2["adj_defense"].apply(lambda x: math.ceil(x))
df2["adj_total"] = df2["adj_attack"] + df2["adj_defense"] + df2["HP"] + df2["Speed"]

In [934]:
# Dejamos solo las columnas de interes
df2.drop(
    ["avg_attack", "avg_defense", "Attack Type", "Defense Type", "Total"],
    axis=1,
    inplace=True,
)

In [935]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   #            800 non-null    int64
 1   Type 1       800 non-null    int64
 2   Type 2       800 non-null    int64
 3   HP           800 non-null    int64
 4   Speed        800 non-null    int64
 5   adj_attack   800 non-null    int64
 6   adj_defense  800 non-null    int64
 7   adj_total    800 non-null    int64
dtypes: int64(8)
memory usage: 50.1 KB


In [936]:
df2.describe()

,#,Type 1,Type 2,HP,Speed,adj_attack,adj_defense,adj_total
count,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,362.813750,7.306250,5.918750,69.258750,68.277500,61.783750,65.350000,264.670000
std,208.343798,4.876955,6.952163,25.534669,29.060474,33.484489,36.001947,89.004865
min,1.000000,1.000000,0.000000,1.000000,5.000000,5.000000,11.000000,92.000000
25%,184.750000,3.000000,0.000000,50.000000,45.000000,36.000000,37.000000,195.000000
50%,364.500000,5.000000,1.000000,65.000000,65.000000,53.000000,55.000000,252.000000
75%,539.250000,11.000000,11.000000,80.000000,90.000000,83.250000,89.000000,331.250000
max,721.000000,18.000000,18.000000,255.000000,180.000000,188.000000,250.000000,531.000000


In [937]:
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,57,57
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,71,72
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,91,92
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,111,122
4,4,Charmander,Fire,None,309,39,52,43,60,50,65,1,False,56,47


In [938]:
df2.head()

,#,Type 1,Type 2,HP,Speed,adj_attack,adj_defense,adj_total
0,1,1,6,45,45,54,66,210
1,2,1,6,60,60,68,83,271
2,3,1,6,80,80,86,107,353
3,3,1,6,80,80,105,141,406
4,4,2,0,39,65,32,32,168


In [939]:
# Convertimos el dataframe a un numpy array
df3 = df2.to_numpy()

### **PCA**

In [940]:
# Aplicamos PCA con las columnas HP, speed, adj_attack y adj_defense
pca = PCA(n_components=1)
pca.fit(df3[:, [3, 4, 5, 6]])
stat_pca = pca.transform(df3[:, [3, 4, 5, 6]])
# Aplicamos PCA al type 1 y type 2
pca = PCA(n_components=1)
pca.fit(df3[:, [1, 2]])
type_pca = pca.transform(df3[:, [1, 2]])
df3_pca = np.concatenate((type_pca, stat_pca), axis=1)

### **Mapper**

In [941]:
mapper = km.KeplerMapper(verbose=1)
# La proyeccion sobre el math score, corresponde a proyectar sobre la primera columna de data
projected_data = mapper.fit_transform(df3_pca, projection=[0, 1])  # Math score

KeplerMapper(verbose=1)
..Composing projection pipeline of length 1:
	Projections: [0, 1]
	Distance matrices: False
	Scalers: MinMaxScaler()
..Projecting on data shaped (800, 2)

..Projecting data using: [0, 1]

..Scaling with: MinMaxScaler()



In [942]:
covering = km.Cover(n_cubes=4, perc_overlap=0.2)
covering

Cover(n_cubes=4, perc_overlap=0.2, limits=None, verbose=0)

In [943]:
G = mapper.map(
    projected_data,
    X=df2,
    cover=covering,
    clusterer=sklearn.cluster.KMeans(n_clusters=2),
)

Mapping on data shaped (800, 8) using lens shaped (800, 2)

Creating 16 hypercubes.

Created 58 edges and 32 nodes in 0:00:00.632905.


In [944]:
G["meta_data"]

{'projection': '[0, 1]',
 'n_cubes': 4,
 'perc_overlap': 0.2,
 'clusterer': 'KMeans(n_clusters=2)',
 'scaler': 'MinMaxScaler()'}

In [945]:
mapper.visualize(
    G,
    title="Example 1 of Mapper on the Sphere dataset",
    color_values=np.array(df2[["adj_total"]]),
    color_function_name="Height of the point",
    node_color_function=np.array(["average", "std", "sum", "max", "min"]),
)

Wrote visualization to: mapper_visualization_output.html


'<!DOCTYPE html>\n<html>\n\n<head>\n  <meta charset="utf-8">\n  <meta name="generator" content="KeplerMapper">\n  <title>Example 1 of Mapper on the Sphere dataset | KeplerMapper</title>\n\n  <link rel="icon" type="image/png" href="http://i.imgur.com/axOG6GJ.jpg" />\n\n  <link href=\'https://fonts.googleapis.com/css?family=Roboto+Mono:700,300\' rel=\'stylesheet\' type=\'text/css\'>\n  <style>* {\n  margin: 0;\n  padding: 0;\n}\n\nhtml, body {\n  height: 100%;\n}\n\nbody {\n  font-family: "Roboto Mono", "Helvetica", sans-serif;\n  font-size: 14px;\n}\n\n#logo {\n  width:  85px;\n  height: 85px;\n}\n\n#display {\n  color: #95A5A6;\n  background: #212121;\n}\n\n#header {\n  background: #111111;\n}\n\n#print {\n  color: #000;\n  background: #FFF;\n}\n\nh1 {\n  font-size: 21px;\n  font-weight: 300;\n  font-weight: 300;\n}\n\nh2 {\n  font-size: 18px;\n  padding-bottom: 20px;\n  font-weight: 300;\n}\n\nh3 {\n  font-size: 14px;\n  font-weight: 700;\n  text-transform: uppercase;\n}\n\nh4 {\n  fo

## **Metodologia**

Para poder obtener los pokemones que usaremos en el equipo, hicimos un mapper el cual separaria los datos acorde a sus estadisticas y sus 2 tipos, estas 2 partes las cuales fueron modificadas para mejor satisfacer lo que buscamos, intentando que la clasificacion de los tipos se adentre mas en el analisis. 
Ademas de los cambios que se realizaron a las estadisticas, y del mapeo de los tipos se hicieron 2 PCAs de un componente, uno de las estadisticas, y el otro de los tipos de pokemones. Estos datos de PCA despues fueron concatenados, y se introducieron al mapper.

El mapper se hizo con la proyeccion de los elementos del PCA, el covering se hizo con 4 cubos y un perc overlap de 0.2 (cercania de los cubos). De clusterer se uso KMeans con 2 clusters, y el color de los nodos se ajustó con el total ajustado que se calculó con las estadísticas modificadas.

### **Miembros del Equipo**

Clusters que se usaran para el primer equipo:

In [946]:
nodeid = "cube3_cluster1"
cluster1 = G["nodes"][nodeid]
eleccion1 = df.loc[cluster1, :]
eleccion1

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
424,383,GroudonPrimal Groudon,Ground,Fire,770,100,180,160,150,90,90,3,True,165,125


In [947]:
nodeid = "cube11_cluster0"
cluster2 = G["nodes"][nodeid]
eleccion2 = df.loc[cluster2, :]
eleccion2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
163,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True,172,100
224,208,SteelixMega Steelix,Steel,Ground,610,75,125,230,55,95,30,2,False,90,163
279,257,BlazikenMega Blaziken,Fire,Fighting,630,80,160,80,130,80,100,3,False,145,80
412,376,Metagross,Steel,Psychic,600,80,135,130,95,90,70,3,False,115,110
413,376,MetagrossMega Metagross,Steel,Psychic,700,80,145,150,105,110,110,3,False,125,130
418,380,LatiasMega Latias,Dragon,Psychic,700,80,100,120,140,150,110,3,True,120,135
420,381,LatiosMega Latios,Dragon,Psychic,700,80,130,100,160,120,110,3,True,145,110
494,445,GarchompMega Garchomp,Dragon,Ground,700,108,170,115,120,95,92,4,False,145,105


In [948]:
nodeid = "cube15_cluster0"
cluster3 = G["nodes"][nodeid]
eleccion3 = df.loc[cluster3, :]
eleccion3

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
409,373,SalamenceMega Salamence,Dragon,Flying,700,95,145,130,120,90,120,3,False,133,110
425,384,Rayquaza,Dragon,Flying,680,105,150,90,150,90,95,3,True,150,90
426,384,RayquazaMega Rayquaza,Dragon,Flying,780,105,180,100,180,100,115,3,True,180,100
498,448,LucarioMega Lucario,Fighting,Steel,625,70,145,88,140,70,112,4,False,143,79
540,483,Dialga,Steel,Dragon,680,100,120,120,150,100,90,4,True,135,110
541,484,Palkia,Water,Dragon,680,90,120,100,150,120,100,4,True,135,110
542,485,Heatran,Fire,Steel,600,91,90,106,130,106,77,4,True,110,106
544,487,GiratinaAltered Forme,Ghost,Dragon,680,150,100,120,100,120,90,4,True,100,120
545,487,GiratinaOrigin Forme,Ghost,Dragon,680,150,120,100,120,100,90,4,True,120,100
711,646,KyuremBlack Kyurem,Dragon,Ice,700,125,170,100,120,90,95,5,True,145,95


In [949]:
nodeid = "cube15_cluster1"
cluster4 = G["nodes"][nodeid]
eleccion4 = df.loc[cluster4, :]
eleccion4

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False,130,98
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False,132,97
141,130,GyaradosMega Gyarados,Water,Dark,640,95,155,109,70,130,81,1,False,113,120
229,212,ScizorMega Scizor,Bug,Steel,600,70,150,140,65,100,75,2,False,108,120
268,248,TyranitarMega Tyranitar,Rock,Dark,700,100,164,150,95,120,71,2,False,130,135
269,249,Lugia,Psychic,Flying,680,106,90,130,90,154,110,2,True,90,142
270,250,Ho-oh,Fire,Flying,680,106,130,90,110,154,90,2,True,120,122


In [950]:
nodeid = "cube7_cluster0"
cluster5 = G["nodes"][nodeid]
eleccion5 = df.loc[cluster5, :]
eleccion5

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
707,644,Zekrom,Dragon,Electric,680,100,150,120,120,100,90,5,True,135,110
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True,100,150
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True,160,110


In [951]:
nodeid = "cube7_cluster1"
cluster6 = G["nodes"][nodeid]
eleccion6 = df.loc[cluster6, :]
eleccion6

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
224,208,SteelixMega Steelix,Steel,Ground,610,75,125,230,55,95,30,2,False,90,163
279,257,BlazikenMega Blaziken,Fire,Fighting,630,80,160,80,130,80,100,3,False,145,80
283,260,SwampertMega Swampert,Water,Ground,635,100,150,110,95,110,70,3,False,123,110
354,323,CameruptMega Camerupt,Fire,Ground,560,70,120,100,145,105,20,3,False,133,103
494,445,GarchompMega Garchomp,Dragon,Ground,700,108,170,115,120,95,92,4,False,145,105


Estos fueron los nodos con mas concentracion de pokemones con estadisticas totales mayores, despues de que fueron agrupados por las estadisticas de HP, Speed, Average Attack, y Average Defense en un PCA, al igual que con otro PCA de los tipos. Eligiremos 2 equipos con los pokemones que nos dieron los 6 nodos seleccionados, uno sin restricciones, el cual incluira una cantidad sin restriccion de pokemones mega, los cuales en un juego normal de pokemon solo se permite 1 por equipo, regla que seguiremos en el segundo equipo. 

### **Equipo 1**

Elegimos un pokemon por equipo, eligiendo el pokemon mas fuerte por cluster, e intentando variar los tipos de los pokemones.

In [952]:
# Elejimos el equipo de 6 pokemones
mon1 = eleccion1.iloc[0]
mon2 = eleccion2.iloc[0]
mon3 = eleccion3.iloc[2]
mon4 = eleccion4.iloc[-3]
mon5 = eleccion5.iloc[-1]
mon6 = eleccion6.iloc[-1]

In [953]:
# Equipo 1
team1 = [mon1, mon2, mon3, mon4, mon5, mon6]


team1 = pd.DataFrame(team1)


team1.reset_index(drop=True, inplace=True)


team1

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
0,383,GroudonPrimal Groudon,Ground,Fire,770,100,180,160,150,90,90,3,True,165,125
1,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True,172,100
2,384,RayquazaMega Rayquaza,Dragon,Flying,780,105,180,100,180,100,115,3,True,180,100
3,248,TyranitarMega Tyranitar,Rock,Dark,700,100,164,150,95,120,71,2,False,130,135
4,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True,160,110
5,445,GarchompMega Garchomp,Dragon,Ground,700,108,170,115,120,95,92,4,False,145,105


### **Equipo 2**

Quitamos  los clusters de 100% Megas, e igual elegimos un pokemon por cluster intentando elegir los mas fuertes en estadisticas totales, y los tipos variando entre cada pokemon.

In [954]:
# Agarramos los mismos clusters solo cambiando el 2 y 6 que tienen los pokemones de puros megas.
eleccion1_2 = eleccion1
nodeid = "cube3_cluster0"
cluster2_2 = G["nodes"][nodeid]
eleccion2_2 = df.loc[cluster2_2, :]
eleccion3_2 = eleccion3
eleccion4_2 = eleccion4
eleccion5_2 = eleccion5
nodeid = "cube11_cluster1"
cluster6_2 = G["nodes"][nodeid]
eleccion6_2 = df.loc[cluster6_2, :]

In [955]:
eleccion1_2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
424,383,GroudonPrimal Groudon,Ground,Fire,770,100,180,160,150,90,90,3,True,165,125


In [956]:
eleccion2_2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
706,643,Reshiram,Dragon,Fire,680,100,120,100,150,120,90,5,True,135,110
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True,120,105


In [957]:
eleccion3_2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
409,373,SalamenceMega Salamence,Dragon,Flying,700,95,145,130,120,90,120,3,False,133,110
425,384,Rayquaza,Dragon,Flying,680,105,150,90,150,90,95,3,True,150,90
426,384,RayquazaMega Rayquaza,Dragon,Flying,780,105,180,100,180,100,115,3,True,180,100
498,448,LucarioMega Lucario,Fighting,Steel,625,70,145,88,140,70,112,4,False,143,79
540,483,Dialga,Steel,Dragon,680,100,120,120,150,100,90,4,True,135,110
541,484,Palkia,Water,Dragon,680,90,120,100,150,120,100,4,True,135,110
542,485,Heatran,Fire,Steel,600,91,90,106,130,106,77,4,True,110,106
544,487,GiratinaAltered Forme,Ghost,Dragon,680,150,100,120,100,120,90,4,True,100,120
545,487,GiratinaOrigin Forme,Ghost,Dragon,680,150,120,100,120,100,90,4,True,120,100
711,646,KyuremBlack Kyurem,Dragon,Ice,700,125,170,100,120,90,95,5,True,145,95


In [958]:
eleccion4_2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False,130,98
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False,132,97
141,130,GyaradosMega Gyarados,Water,Dark,640,95,155,109,70,130,81,1,False,113,120
229,212,ScizorMega Scizor,Bug,Steel,600,70,150,140,65,100,75,2,False,108,120
268,248,TyranitarMega Tyranitar,Rock,Dark,700,100,164,150,95,120,71,2,False,130,135
269,249,Lugia,Psychic,Flying,680,106,90,130,90,154,110,2,True,90,142
270,250,Ho-oh,Fire,Flying,680,106,130,90,110,154,90,2,True,120,122


In [959]:
eleccion5_2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
707,644,Zekrom,Dragon,Electric,680,100,150,120,120,100,90,5,True,135,110
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True,100,150
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True,160,110


In [960]:
eleccion6_2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True,100,150
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True,160,110


In [961]:
# Eleccion del equipo de 6 pokemones
mon1_2 = eleccion1_2.iloc[0]
mon2_2 = eleccion2_2.iloc[1]
mon3_2 = eleccion3_2.iloc[2]
mon4_2 = eleccion4_2.iloc[-2]
mon5_2 = eleccion5_2.iloc[0]
mon6_2 = eleccion6_2.iloc[0]

In [962]:
# Equipo 2
team2 = [mon1_2, mon2_2, mon3_2, mon4_2, mon5_2, mon6_2]
team2 = pd.DataFrame(team2)


team2.reset_index(drop=True, inplace=True)
team2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,avg_attack,avg_defense
0,383,GroudonPrimal Groudon,Ground,Fire,770,100,180,160,150,90,90,3,True,165,125
1,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True,120,105
2,384,RayquazaMega Rayquaza,Dragon,Flying,780,105,180,100,180,100,115,3,True,180,100
3,249,Lugia,Psychic,Flying,680,106,90,130,90,154,110,2,True,90,142
4,644,Zekrom,Dragon,Electric,680,100,150,120,120,100,90,5,True,135,110
5,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True,100,150


## **Resultados**

### **Equipos Finales**

### Equipo 1:

Pokemon 1: **Primal Groudon** Total Stats: 770

Tipo 1: Fire 

Tipo 2: Ground


Pokemon 2: **Mega Mewtwo X** Total Stats: 780

Tipo 1: Psychic

Tipo 2: Fighting


Pokemon 3: **Mega Rayquaza** Total Stats: 780

Tipo 1: Dragon

Tipo 2: Flying


Pokemon 4: **Mega Tyranitar** Total Stats: 700 

Tipo 1: Rock

Tipo 2: Dark


Pokemon 5: **Mega Diancie** Total Stats: 700

Tipo 1: Rock

Tipo 2: Fairy


Pokemon 6: **Mega Garchomp** Total Stats: 700

Tipo 1: Dragon

Tipo 2: Ground



### Equipo 2:

Pokemon 1: **Primal Groudon** Total Stats: 770

Tipo 1: Fire 

Tipo 2: Ground


Pokemon 2: **Volcanion** Total Stats: 600

Tipo 1: Fire 

Tipo 2: Water


Pokemon 3: **Mega Rayquaza** Total Stats: 780

Tipo 1: Dragon

Tipo 2: Flying


Pokemon 4: **Lugia** Total Stats: 680 

Tipo 1: Psychic

Tipo 2: Flying


Pokemon 5: **Zekrom** Total Stats: 680

Tipo 1: Dragon

Tipo 2: Electric


Pokemon 6: **Diancie** Total Stats: 600

Tipo 1: Rock 

Tipo 2: Fairy



Aqui podemos ver que elegimos pokemones con stats muy altas, el primer equipo siendo puros pokemones con mega evolucion, y Primal Groudon el cual es una forma mejorada de Groudon normal. Tambien se puede ver la variedad de tipos en los pokemones. El equipo se ve que bajan las estadisticas al tener que restringirnos a solo un pokemon mega, pero siguen siendo estadisticas altas en el espectro pokemon general.